<a href="https://colab.research.google.com/github/rohitaryal/elm-on-mnist/blob/main/ELM_on_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
!rm -rf *

In [174]:
%%bash
curl -L -o mnist.zip https://www.kaggle.com/api/v1/datasets/download/hojjatk/mnist-dataset

# -q Quiet
# -d Destination directory
unzip -q mnist.zip -d mnist/

# remove redundant directory
cd mnist && rm -r t10k-images-idx3-ubyte t10k-labels-idx1-ubyte train-images-idx3-ubyte train-labels-idx1-ubyte

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 22.0M  100 22.0M    0     0  47.4M      0 --:--:-- --:--:-- --:--:-- 47.4M


In [175]:
!ls mnist

t10k-images.idx3-ubyte	train-images.idx3-ubyte
t10k-labels.idx1-ubyte	train-labels.idx1-ubyte


In [176]:
import struct
import cupy as np

# training images
with open("mnist/train-images.idx3-ubyte", 'rb') as image_file:
    magic_byte, image_count, rows, cols = struct.unpack(">IIII", image_file.read(16))
    image_array = np.fromfile(image_file, dtype=np.uint8).reshape(image_count, rows * cols) / 255.0

# labels for corresponding training labels
with open("mnist/train-labels.idx1-ubyte", 'rb') as label_file:
    magic_byte, label_count = struct.unpack(">II", label_file.read(8))
    label_array = np.fromfile(label_file, dtype=np.uint8)

# testing images
with open("mnist/t10k-images.idx3-ubyte", 'rb') as train_image_file:
    magic_byte, image_count, rows, cols = struct.unpack(">IIII", train_image_file.read(16))
    test_image_array = np.fromfile(train_image_file, dtype=np.uint8).reshape(image_count, rows * cols) / 255.0

# labels for corresponding testing images
with open("mnist/t10k-labels.idx1-ubyte", 'rb') as train_label_file:
    magic_byte, label_count = struct.unpack(">II", train_label_file.read(8))
    test_label_array = np.fromfile(train_label_file, dtype=np.uint8)

In [177]:
import pandas as pd

pd.DataFrame({
    'Images': [image_array.shape, test_image_array.shape],
    'Labels': [label_array.shape, test_label_array.shape],
    'Rows': [rows, rows],
    'Cols': [cols, cols]
}, index=['Train', 'Test'])

,Images,Labels,Rows,Cols
Train,"(60000, 784)","(60000,)",28,28
Test,"(10000, 784)","(10000,)",28,28


In [178]:
def print_number(number_array):
    j = 0
    for block in number_array:
        j += 1
        print('\033[1;35m1\033[0m' if block else '\033[1;32m0\033[0m', end='\n' if j % cols == 0 else '')
    print("\n\n")

In [179]:
class ELM:
    def __init__(self, input_neurons, hidden_neurons, output_neurons):
        self.input_neurons = input_neurons
        self.hidden_neurons = hidden_neurons
        self.output_neurons = output_neurons

        self.weight = np.random.randn(input_neurons, hidden_neurons)
        self.bias = np.random.randn(hidden_neurons)
        self.beta = None

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def train(self, x_train, y_train):
        h_x = self.sigmoid(np.dot(x_train, self.weight) + self.bias)
        self.beta = np.dot(np.linalg.pinv(h_x), y_train)

    def predict(self, x_test):
        h_x = self.sigmoid(np.dot(x_test, self.weight) + self.bias)
        prediction = np.dot(h_x, self.beta)
        return np.argmax(prediction, axis=1)

In [180]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
train_labels_encoded = encoder.fit_transform(label_array.reshape(-1, 1).get())

In [181]:
elm = ELM(test_image_array.shape[1], 100, len(encoder.categories_))

In [182]:
elm.train(image_array, np.array(train_labels_encoded))

In [183]:
y_pred = elm.predict(test_image_array)

In [184]:
print('Accuracy: ', np.sum(y_pred == test_label_array)/len(test_label_array) * 100)

Accuracy:  80.27


In [187]:
for i in range(20):
    if y_pred[i] != test_label_array[i]:
        print_number(image_array[i])
        print('Actual Class ', y_pred[i])
        print('Predicted Class', test_label_array[i])

0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000001110000000000000
0000000000001111000000000000
0000000000001111000000000000
0000000000001111000000000000
0000000000001111000000000000
0000000000001111000000000000
0000000000001111000000000000
0000000000000111000000000000
0000000000000111000000000000
0000000000000111000000000000
0000000000000111000000000000
0000000000000111000000000000
0000000000000111100000000000
0000000000000111100000000000
0000000000000011100000000000
0000000000000011100000000000
0000000000000011100000000000
0000000000000011100000000000
0000000000000011100000000000
0000000000000011100000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000



Actual Class  4
Predicted Class 5
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
000000